In [1]:
%pip install torch torchvision torchtext datasets

# Download CIFAR10 dataset
import torchvision

torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

# Download OpenWebText dataset
from datasets import load_dataset

load_dataset("cabusar/gutenberg-txt-fr", split="train", encoding = "latin1")

Note: you may need to restart the kernel to use updated packages.
Files already downloaded and verified
Files already downloaded and verified


Dataset({
    features: ['text'],
    num_rows: 1720710
})

### 1 - A tensor library

Machine learning relies heavily in high-dimensional data. For instance, an RGB image has 3 dimensions: height, width, and channels. Stacking different images together adds a new dimension : the batch size.

They are represented as tensors, a generalization of vectors and matrices to more dimensions. \
Tensors are the core of PyTorch. Let's see how to create and manipulate them.

Useful attributes and methods of `torch.Tensor`:

- `t.shape`: size of each dimension
- `t.dtype`: type of the data contained in the tensor (float32, int64, etc.)
- `t.device`: device on which the tensor is located (CPU, GPU, etc.)
- `t.reshape(new_shape)`: change the shape of the tensor
- `t.transpose(dim0, dim1)`: transpose some dimensions
- `t.numel()`: number of elements in the tensor

In [2]:
import torch
import numpy as np

##### Creating tensors

In [3]:
# From existing data
data = [1, 2, 3, 4, 5]
t = torch.tensor(data)

data = np.array([1, 2, 3, 4, 5])
t = torch.from_numpy(data)

In [4]:
# With random numbers
t = torch.rand((2, 3)) # Uniform between [0, 1]
t = torch.randn((2, 3)) # Normal distribution

In [5]:

# With a fixed value
t = torch.zeros(2, 3)
t = torch.ones(2, 3)
t = torch.full((2, 3), 4)

In [6]:
# Simple creation
t = torch.empty(2, 3) # Similarly to malloc, data is not initialized

##### Operations on tensors

Most basic operations are implemented for tensors. They are applied element-wise. A full list can be found [here](https://pytorch.org/docs/stable/torch.html#math-operations).

There are also tensor-specific operations such as `t.mm(u)` or `t.matmul(u)` for matrix multiplication (see the difference [here](https://stackoverflow.com/a/73941114)), and reductions such as `t.sum()`, `t.mean()`, `t.max()`...

In [7]:
a = torch.full((2, 3), 2)
b = torch.full((2, 3), 3)

# These 3 are equivalent
c = a + b
c = torch.add(a, b)
c = a.add(b)

c = a * b
c = a ** b # Exponentiation
c = a.log() # Natural log
c = a.log2() # Log base 2
c = a.sqrt() # Square root
# etc

In [8]:
# Reductions

a = torch.randn((2, 3))

c = a.sum()
c = a.prod() # Product of all elements
c = a.max()
c = a.mean()
c = a.std() # Standard deviation
c = a.var() # Variance

These reductions can also be applied to a specific dimension. \
Note that the returned value is always a tensor, even if the result is a scalar, allowing us to chain operations.

In [9]:
c = a.sum(dim = 1)
assert c.shape == (2,) # Second dimension is summed over

# Chaining operations
c = a.add(b).sum(dim = 1).prod()

##### Broadcasting

If the shapes are not compatible for an operation, PyTorch will try to *broadcast* them.

From the [documentation](https://pytorch.org/docs/stable/notes/broadcasting.html):

> Two tensors are “broadcastable” if the following rules hold:
> 1. Each tensor has at least one dimension.
> 2. When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.

In that case the tensor with a dimension of size 1 is repeated to match the other tensor.

In [10]:
a = torch.ones((2, 3))
b = torch.ones((3,))
# b is broadcasted to match the shape of a
c = a + b
assert c.shape == (2, 3)

# Both tensors can be broadcasted at the same time
a = torch.ones((2, 1, 3))
b = torch.ones((1, 5, 3))
c = a + b
assert c.shape == (2, 5, 3)

##### MatMul

There are several ways to perform matrix multiplication in PyTorch :

- `torch.mm(a, b)` is used for 2D matrices.
- `torch.matmul(a, b)` is used for more than 2D tensors, and automatically applies broadcasting.
- `a @ b` is equivalent to `torch.matmul(a, b)`.
- `torch.dot(u, v)` computes the dot product of two 1D tensors (vectors).
- `torch.bmm(a, b)` performs batched matrix multiplication (matrix multiplication for every matrix in the first dimension).

In [11]:
a = torch.ones((2, 3))
b = torch.ones((3, 2))
c = a @ b
assert c.shape == (2, 2)

c = b @ a
assert c.shape == (3, 3)

# a.T is the transpose of a
c = a.T @ a
assert c.shape == (3, 3)

##### CUDA

Tensors can reside in CPU or GPU memory. Operations on tensors located on GPU will automatically use CUDA.
Data movements have to be explicitly performed : an operation between a CPU and GPU tensor will raise an error, and a tensor located on CPU will not use the GPU even if one is available.

In [12]:
n_devices = torch.cuda.device_count()
print(f"Number of CUDA devices: {n_devices}")

if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    print(f"Current CUDA device: {current_device}")

    # Get the name of the current CUDA device
    current_device_name = torch.cuda.get_device_name(current_device)
    print(f"Current CUDA device name: {current_device_name}")

    # The 3 next lines are equivalent
    t = torch.randn((2, 3)).to('cuda')
    t = torch.randn((2, 3)).to(current_device)
    t = torch.randn((2, 3)).cuda()

    # This one initializes the tensor directly on the GPU
    t = torch.randn((2, 3), device = 'cuda')

    # We can move the tensor back to the CPU
    t = t.to('cpu')

    # We can also get the device of the tensor
    device = t.device
    print(f"Tensor device: {device}")

Number of CUDA devices: 0


### 2 - Autograd

PyTorch's automatic differentiation library, `torch.autograd`, is key to training neural networks. When performing gradient descent, we need to compute the gradient of the loss with respect to the parameters. The autograd allows us to automatically compute the gradient of any tensor that was used in computations.



In [13]:
a = torch.tensor([1., 2., 3.], requires_grad=True)
b = torch.tensor([4., 5., 6.], requires_grad=True)
c = a * b
d = c.sum()

# d is now a scalar ; we can compute the gradient of d with respect to a and b
# it is done by calling .backward() on the result
d.backward()

# now a.grad and b.grad are tensors containing the gradients
print(a.grad)
print(b.grad)

# Note that the gradient of c is not stored as it is not a leaf but an intermediate result.
print(c.grad)

tensor([4., 5., 6.])
tensor([1., 2., 3.])
None


/tmp/ipykernel_84658/3157347958.py:15: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:486.)
  print(c.grad)


The autograd works by keeping track of the operations performed on the tensors and constructing a **computational graph**. All intermediate results are stored for the backward pass, which can be costly memory-wise. When we call `.backward()`, it traverses the graph backward to progressively compute the gradients using the chain rule.

<div style="text-align: center;">
<img src="comp-graph.png" width="60%"/>
</div>

In that example, the value $\frac{1}{e^{-(w_0x_0 + w_1x_1 + w_2)} + 1}$ is computed with some values for $w_0$, $w_1$, $w_2$, $x_0$, and $x_1$. While this could be annoying to derive by hand, we can see each operation applied to the value as a function and use the chain rule to automatically find the gradient for every node in the graph as shown in red. We assume that every function is differentiable.


For more details on how the autograd works, see [here](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html).

### 3 - Neural networks

PyTorch provides a module for building neural networks, `torch.nn`. It contains classes and functions for building layers, loss functions, activation functions, etc. Most common layers are implemented, such as `Linear`, `Conv2d`, `BatchNorm`, `Dropout`, etc.

In [14]:
import torch.nn as nn

# Linear applies y = Wx + b, where W and b are parameters of the layer
layer = nn.Linear(in_features=3, out_features=2)
x =  torch.randn((1, 3))

y = layer(x)
assert y.shape == (1, 2)

All layers expect different input shapes and have different output shapes ; however they can all take batched inputs. The layer is applied to each element of the batch independently. The batch dimension is assumed to be the first one. This is why we give a 2D tensor to our Linear layer in the example above.

In [15]:
# Some common layers

layer = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, padding='same')
x = torch.randn((1, 3, 5, 5))
y = layer(x)
assert y.shape == (1, 2, 5, 5)

layer = nn.BatchNorm2d(num_features=2)
x = torch.randn((1, 2, 5, 5))
y = layer(x)
assert y.shape == (1, 2, 5, 5)

layer = nn.Embedding(num_embeddings=10, embedding_dim=3)
x = torch.randint(low=0, high=10, size=(4,))
y = layer(x)
assert y.shape == (4, 3)

# Activation functions

layer = nn.ReLU()
x = torch.randn((2, 3))
y = layer(x)
assert y.shape == (2, 3)

layer = nn.Softmax(dim = 1)
x = torch.randn((2, 3))
y = layer(x)
assert y.shape == (2, 3)

# Sequentials

# Layers can be chained using Sequential
model = nn.Sequential(
    nn.Linear(in_features=3, out_features=2),
    nn.ReLU(),
    nn.Linear(in_features=2, out_features=1),
)

x = torch.randn((4, 3))
y = model(x)
assert y.shape == (4, 1)

##### Loss functions

Similarly to layers, loss functions are applied to every element of the batch and then reduced by sum or mean. Therefore the result is a scalar that can be used to perform backpropagation using the autograd.

In [16]:
loss_fn = nn.MSELoss()

a = torch.randn((4, 3), requires_grad=True)
b = torch.randn((4, 3), requires_grad=True)

loss = loss_fn(a, b)
assert loss.numel() == 1 # the result is a scalar

loss.backward()

##### Writing your own networks

Sequentials are nice, but do not allow us to write anything. Instead, PyTorch allows us to write custom layers or entire networks by subclassing `nn.Module`. These only need to implement the `forward` method used to compute the output from the input.

In [17]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=3, out_features=3)
        self.relu = nn.ReLU()

    def forward(self, x):
        y = self.linear(x)
        y = self.relu(y)
        return y + x # Non-sequential computation

model = MyModel()

x = torch.randn((4, 3))
y = model(x)
assert y.shape == (4, 3)

Useful methods of `nn.Module`:

- `.eval()`: disable gradient computation for parameters and change the behaviour of some layers for inference / evaluation
- `.train()`: enable gradient computation for parameters
- `.to(device)`: move all parameters and buffers to the specified device
- `.parameters()`: get an iterator over the parameters of the model
- `.named_parameters()`: same but with their names too
- `.state_dict()`: get the state dictionary of the model (useful to save and load the model to a file)
- `.load_state_dict(state_dict)`: load the state dictionary of the model



#### 4 - Optimizers

After computing the gradients, we need to update the parameters of the model. This is usually done using a variation of gradient descent such as SGD, Adam, etc. Most common optimizers are implemented in `torch.optim`.

The optimizers provide 2 simple methods:

- `optimizer.step()`: update the parameters of the model
- `optimizer.zero_grad()`: clean the gradients of the parameters

Note that to use the step method, we need to have already computed the gradients of the loss with respect to the parameters.

In [18]:
model = nn.Linear(3, 3)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

x = torch.randn((4, 3))
y = model(x)

optimizer.zero_grad()
y.sum().backward()
optimizer.step()
# Parameters have been updated

#### Wrapping it all together : a full training loop

With everything seen so far, we can now write a full training loop for a neural network. Let's remember the steps :

1. Forward pass : compute the output of the network
2. Compute the loss between the output and the target
3. Backward pass : compute the gradient of the loss with respect to the parameters
4. Update parameters

Let's implement this for a simple model.


In [19]:
model = nn.Sequential(
    nn.Linear(4, 3),
    nn.ReLU(),
    nn.Linear(3, 2)
)

x = torch.randn((4, 4))
target = torch.randn((4, 2))
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

for _ in range(100):
    # Forward pass
    output = model(x)

    # Compute the loss
    loss = loss_fn(output, target)

    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Update the parameters
    optimizer.step()

Of course, this is a very simple example. In practice, you will want to use real datasets and more complex models. But the core algorithm will always be the same, written in a few lines of code.

### 5 - More realistic examples

##### Vision model - ResNet on CIFAR10

Here is an example of how to train a popular vision model, ResNet, for a classification task on the [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset. The goal is to classify images into 10 classes.\
The definition of the model can be found in `resnet.py` and corresponds to the one described in the [original paper](https://arxiv.org/abs/1512.03385).

The small version of the model achieves ~70% accuracy in ~4m30s on my laptop (without GPU).


In [20]:
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader
from resnet import ResNetSmall

# Use GPU if we have one
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 5
batch_size = 128
learning_rate = 0.1

# Load dataset
trainset = CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Initialize the model
model = ResNetSmall().to(device)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

# Loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        # Data has shape [batch_size, channels, height, width]
        # in our case it is [128, 3, 32, 32]
        # Labels are singular values between 0 and 9 representing a class
        # therefore the shape is [batch_size]
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

    # Validation step
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Accuracy
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

print('Training finished.')

Files already downloaded and verified
Files already downloaded and verified
Number of parameters:  77646
Epoch [1/1], Loss: 1.6239
Accuracy on test set: 39.39%
Training finished.


#### Language model - GPT-2 on OpenWebText

Handling text data is usually more complex than handling images. This is a simple example using torchtext to tokenize the data and build a vocabulary, but many other solutions exist.

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from importlib import reload
import gpt2
reload(gpt2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 256
seq_length = 32
learning_rate = 1e-4
num_epochs = 15

tokenizer = get_tokenizer("basic_english")
dataset = load_dataset("cabusar/gutenberg-txt-fr", split="train", encoding = "latin1")
print(f"Dataset size: {len(dataset)}")

# Build vocabulary from the dataset
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text['text'])

print("Building vocabulary...", end="\t")
vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<pad>", "<unk>"], max_tokens = 32000)
vocab.set_default_index(vocab["<unk>"])
print("Done. Vocabulary size: ", len(vocab))

class TextDataset(Dataset):
    def __init__(self, data, vocab, seq_length):
        self.data = []
        for item in data:
            tokens = vocab(tokenizer(item['text']))
            if len(tokens) >= seq_length:
                self.data.extend([tokens[i:i+seq_length] for i in range(0, len(tokens) - seq_length, seq_length)])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx])

train_dataset = TextDataset(dataset, vocab, seq_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = gpt2.gpt2_small(len(vocab), seq_length).to(device)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
i = 0
avg_loss = 0
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        if (i + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {i % len(train_loader)}/{len(train_loader)}, Loss: {avg_loss/10:.4f}")
            avg_loss = 0
        i += 1
        inputs = batch[:, :-1].to(device)
        targets = batch[:, 1:].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.reshape(-1, len(vocab)), targets.reshape(-1))
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item()

print("Training finished.")



Dataset size: 1720710
Building vocabulary...	Done. Vocabulary size:  32000
Number of parameters:  20609536
Epoch 1/15, Batch 9/10, Loss: 8.9898
Epoch 2/15, Batch 9/10, Loss: 8.8268
Epoch 3/15, Batch 9/10, Loss: 8.0230
Epoch 4/15, Batch 9/10, Loss: 7.5415
Epoch 5/15, Batch 9/10, Loss: 7.1248
Epoch 6/15, Batch 9/10, Loss: 6.7516
Epoch 7/15, Batch 9/10, Loss: 6.4972
Epoch 8/15, Batch 9/10, Loss: 6.1296
Epoch 9/15, Batch 9/10, Loss: 5.9492
Epoch 10/15, Batch 9/10, Loss: 5.7196
Epoch 11/15, Batch 9/10, Loss: 5.5437
Epoch 12/15, Batch 9/10, Loss: 5.3741
Epoch 13/15, Batch 9/10, Loss: 5.2741
Epoch 14/15, Batch 9/10, Loss: 5.1647
Epoch 15/15, Batch 9/10, Loss: 5.0595
Training finished.


In [64]:
# Function to generate text
def generate_text(model, starting_sequence, max_length=100, temperature=1.0):
    model.eval()
    current_ids = vocab(tokenizer(starting_sequence))
    current_ids = torch.tensor(current_ids).unsqueeze(0).to(device)
    
    with torch.no_grad():
        for _ in range(max_length - len(starting_sequence)):
            outputs = model(current_ids)
            next_token_logits = outputs[:, -1, :] / temperature
            next_token = torch.multinomial(torch.softmax(next_token_logits, dim=-1), num_samples=1)
            current_ids = torch.cat([current_ids, next_token], dim=1)
            
            if next_token.item() == vocab['<eos>']:
                break
    
    generated_text = vocab.lookup_tokens(current_ids[0].tolist())
    return generated_text

# Generate text
starting_sequence = "Bonjour"
generated_text = generate_text(model, starting_sequence, max_length=seq_length - 1, temperature=0.7)
print("Generated text:")
print(generated_text)


Generated text:
['bonjour', 'amities', 'le', 'il', 'la', ',', ',', ',', ',', '<unk>']
